In [1]:
import re
import math
import csv
import sys
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from obspy import UTCDateTime, Stream, Trace,read
import torch
import torch.nn.functional as F
import model_cnn_cls
import pandas as pd
import pygmt

In [2]:
path="/home/sysop/share/drive-download-20220316T143533Z-001"
#path="/home/sysop/share/2021_10_felt"

In [3]:
def batch_showname(path):
    afile_name=[]
    pfile_name=[]
    for fname in os.listdir(path):
        if fname[-3]=="A":
            afile_name.append(fname)
        if fname[-3]=="P":
            pfile_name.append(fname)
    return afile_name,pfile_name

In [60]:
def unpackAfile(infile):

# == opening Afile ==
    b = os.path.getsize(infile)
    FH = open(infile, 'rb')
    line = FH.read(b)
    fileHeader = struct.unpack("<4s3h6bh6s", line[0:24])

    fileLength = fileHeader[3]
    port = fileHeader[10]
    # FirstStn = fileHeader[11][0:4].decode('ASCII').rstrip()
# =================================Header=================================

    portHeader = []
    for i in range(24, port * 32, 32):
        port_data = struct.unpack("<4s4s3sbh2b4s12b", line[i:i+32])
        portHeader.append(port_data)

# =================================Data===================================

    dataStartByte = 24+int(port)*32
    dataPoint = 3*int(port)*int(fileLength)*100
    times = int(port)*3*4
    data = []

    data = struct.unpack("<%di" % dataPoint, line[dataStartByte:dataStartByte + dataPoint*4])


    portHeader = np.array(portHeader)
    data = np.array(data)
    idata = data.reshape((3,port,fileLength*100),order='F')

#== write to obspy Stream --
    sttime = UTCDateTime(fileHeader[1], fileHeader[4], fileHeader[5], fileHeader[6], fileHeader[7], fileHeader[8], fileHeader[2])
    npts = fileHeader[3]*fileHeader[9]
    samp = fileHeader[9]
    afst = Stream()
    
    for stc in range(fileHeader[10]):
        stn = portHeader[stc][0].decode('ASCII').rstrip()
        instrument = portHeader[stc][1].decode('ASCII').rstrip()
        loc = '0'+str(portHeader[stc][6].decode('ASCII'))
        net = str(portHeader[stc][7].decode('ASCII')).rstrip()
        GPS = int(portHeader[stc][3])
        
        # remove GPS unlock or broken station
        if ( GPS == 1 or GPS == 2 ):
            chc = 0
            if instrument == 'FBA':
                chc = 1
            elif instrument == 'SP':
                chc = 4
            elif instrument == 'BB':
                chc = 7
            
            for ch in range(3):
                chn = 'Ch'+str(chc+ch)
                
                stats = {'network': net, 'station': stn, 'location': loc,
                        'channel': chn, 'npts': npts, 'sampling_rate': samp,
                        'starttime': sttime}
                
                data = np.array(idata[ch][stc], dtype=float)
                sttmp = Stream([Trace(data=data, header=stats)])
                afst += sttmp

    return afst

def unpackPfile(infile):
    
    with open(infile) as f:
        lines = f.readlines()
    
    tmp = lines[0]
    year = int(tmp[1:5])
    month = int(tmp[5:7])
    day = int(tmp[7:9])
    hour = int(tmp[9:11])
    minute = int(tmp[11:13])
    sec = float(tmp[13:19])
    
    lat_d = float(tmp[19:21])
    lat_m = float(tmp[21:26])
    
    lon_d = float(tmp[26:29])
    lon_m = float(tmp[29:34])
    dep=float(tmp[34:40])
    dt = datetime(year,month,day,hour,minute,int(sec//1),int(sec%1 * 1000000))
    mag = float(tmp[40:44])

    pfile_info = {}
    pfile_info["ori_time"] = dt
    pfile_info["mag"] = mag
    pfile_info["lat"] = lat_d + lat_m/60.0
    pfile_info["lon"] = lon_d + lon_m/60.0
    pfile_info["dep"] = dep
    
    intensity = {}
    arrival_time = {}
    weighting = {}
    pga = {}
    for i in lines[1:]:
        sta = i[:5].strip() # strip 去掉左右空格
        weighting[sta] = int(float(i[35:39]))
        if i[76:77]==" ":
            intensity[sta] = int(0)
        else:
            intensity[sta] = int(i[76:77])
        if i[78:83]=="     ":
            pga[sta] = int(0)
        else:
            pga[sta] = (i[78:83])
        arrival_time[sta] = pfile_info["ori_time"].replace(minute=int(i[21:23]),second=0,microsecond=0) + timedelta(seconds=float(i[23:29]))
        arrival_time[sta].microsecond * 10**-6
    pfile_info["intensity"] = intensity
    pfile_info["arrival_time"] = arrival_time

    pfile_info["weighting"] = weighting
    pfile_info["pga"] = pga
    
    return pfile_info

In [5]:
def apfile_read(pfile_name,afile_name):
    Pfile = unpackPfile(path+"/"+pfile_name)
    Afile = unpackAfile(path+"/"+afile_name)
    Afile_se = Afile.select(channel="*1")
    Afile_se += Afile.select(channel="*2")
    Afile_se += Afile.select(channel="*3")
    
    e_latlon = [Pfile["lat"],Pfile["lon"]]
    
    return Pfile,Afile,Afile_se,e_latlon

In [7]:
def factor():
    with open("nsta24.dat","r") as f:
        data_nsta=f.readlines()
        
    data_factor={}
    sta_lon_lat={}
    for i in data_nsta:
        i=i.strip()
        if i[34:37]=="CWB" and i[39:42]=="SMT" and i[45:48]=="FBA" and i[100:]=="99999999":
            data_factor[i[0:4].strip()]=[i[49:59],i[60:70],i[71:81]]
            sta_lon_lat[i[0:4].strip()]=[i[5:13],i[14:21]]
            
    return data_factor,sta_lon_lat



In [8]:
def cal_factor(Afile_se,data_factor):
    Afile_se_data=Stream()
    for i in Afile_se.copy():
        i.detrend()
        if i.stats.network == "SMT":
            if i.stats.channel == "Ch1":
                if i.stats.station in data_factor.keys():
                    i.data=i.data*float(data_factor[i.stats.station][0])
                    Afile_se_data.append(i)
            if i.stats.channel == "Ch2":
                if i.stats.station in data_factor.keys():
                    i.data=i.data*float(data_factor[i.stats.station][1])
                    Afile_se_data.append(i)
            if i.stats.channel == "Ch3":
                if i.stats.station in data_factor.keys():
                    i.data=i.data*float(data_factor[i.stats.station][2])
                    Afile_se_data.append(i)
    
    return Afile_se_data

In [9]:
def cut_st(Afile_se_data,Pfile):
    P_arr_time = Pfile["arrival_time"]
    Afile_se_5s=Stream()
    sta_P1=[]
    for i in Afile_se_data:
        if i.stats.station in P_arr_time.keys():
            sta_P1.append(i.stats.station)
            Afile_se_5s.append(i.slice(UTCDateTime(P_arr_time[i.stats.station]),UTCDateTime(P_arr_time[i.stats.station])+4.99))
    sta_P = np.unique(sta_P1).tolist()
    return Afile_se_5s,sta_P

In [10]:
def append_array(Afile_se_5s,sta_P):
    Afile_se_5s_data_123={}
    for i in sta_P:
        data_list_123=[]
        for k in Afile_se_5s:
            if i == k.stats.station:
                data_list_123.append(k.data)
        data_list_123 = np.unique(data_list_123,axis=0)
        if data_list_123 != []:
            Afile_se_5s_data_123[i]=np.array(data_list_123)
    
    return Afile_se_5s_data_123

In [11]:
def data_pga(Afile_se_5s_data_123,Pfile):
    pga_data={}
    for i in Afile_se_5s_data_123:
        if len(Pfile["pga"][i]) > 0:
            pga_data[i]=float(Pfile["pga"][i])
        else:
            pga_data[i]=0
    return pga_data

In [62]:
def result_txt(Pfile,afile_name,result):
    #輸出result為txt檔
    file = open(afile_name+'.txt', 'w')
    file.write(str(Pfile["ori_time"])+' '+str(Pfile["mag"])+' '+str(Pfile["lat"])+' '+str(Pfile["lon"])+'%6s'%str(Pfile["dep"])+'\n')
    for k,v in result.items():
        file.write('%5s'%str(k)+' '+str(v[0])+' '+str(v[1])+' '+'%-11s'%str(v[2])+' '+'%-9s'%str(v[3])+' '+str(v[4])+' '+str(v[5])+'\n')
    file.close()

In [58]:
def data_read(afile_name,result,e_latlon): 
    #從txt讀取資料
    data = f"/home/sysop/share/{afile_name}.txt"
    data_df = pd.read_table(data,header = None,sep = "\s+",skiprows=[0])
    data_df.columns = ["station","lon","lat","probability_X","probability_O","predict","pga"]
    
    data_df_TN = data_df[(data_df["predict"] == 0) & (data_df["pga"]<80)]
    data_df_TN = data_df_TN.reset_index(drop=True)
    data_df_FN = data_df[(data_df["predict"] == 0) & (data_df["pga"]>=80)]
    data_df_FN = data_df_FN.reset_index(drop=True)
    data_df_TP = data_df[(data_df["predict"] == 1) & (data_df["pga"]>=80)]
    data_df_TP = data_df_TP.reset_index(drop=True)
    data_df_FP = data_df[(data_df["predict"] == 1) & (data_df["pga"]<80)]
    data_df_FP = data_df_FP.reset_index(drop=True)
    
    return data_df_TN,data_df_FN,data_df_TP,data_df_FP

In [63]:
def plot_result(data_df_TN,data_df_FN,data_df_TP,data_df_FP,e_latlon,afile_name):
    fig = pygmt.Figure()
    fig.coast(
        projection  = 'N12c',                         # 投影法とサイズ．メルカトルなら 'M12c' など．
        region      = (119, 123, 21, 26),                 # 領域はリストかタプルで与える．
        shorelines  = 'default,black',                    # 海岸線のペンの設定．
        area_thresh = 100,                                # 細かい領域の描画下限 (km^2)
        resolution  = 'f',                                # 'c', 'l', 'i', 'h', 'f' の順に高くなる
        land        = '230/240/220',                      # 陸地の色
        water       = '220/235/250',                      # 水の色
        map_scale   = '100/32/32/400',                  # 距離スケールの水平・縦位置と距離サイズ
        frame       = ['WSen+t"predict_result_80"', 'xafg', 'yafg']  # 南西に軸名・タイトル指定（+t) およびXY方向それぞれの軸情報
    )  
    #-------------------------------
    #TN
    if len(data_df_TN) != 0:
        fig.plot(
            x=data_df_TN.lon,
            y=data_df_TN.lat,
            style="t0.5c",
            color="blue",
            pen="green",
            label=f"TN:{len(data_df_TN)}"
            )
    #-------------------------------
    #FN
    if len(data_df_FN) != 0:
        fig.plot(
            x=data_df_FN.lon,
            y=data_df_FN.lat,
            style="s0.5c",
            color="red",
            pen="green",
            label=f"FN:{len(data_df_FN)}"
            )
    #-------------------------------
    #TP
    if len(data_df_TP) != 0:
        fig.plot(
            x=data_df_TP.lon,
            y=data_df_TP.lat,
            style="s0.5c",
            color="blue",
            pen="green",
            label=f"TP:{len(data_df_TP)}"
            )
    #-------------------------------

    if len(data_df_FP) != 0:
        fig.plot(
            x=data_df_FP.lon,
            y=data_df_FP.lat,
            style="t0.5c",
            color="red",
            pen="green",
            label=f"FP:{len(data_df_FP)}"
            )
    #-------------------------------
    
    
    fig.plot(
        x=e_latlon[1],
        y=e_latlon[0],
        style="a0.5c",
        color="yellow",
        pen="green",
        label="Epicenter"
        )

    fig.legend(position="JBl+jBL+o0.4c")
    fig.show()
    fig.savefig(f"{afile_name}_predict_result_80.jpg")    

class model_cnn():
    def __init__(self, model=None, param_path=None):
        self.model = model
        checkpoint = torch.load(param_path, map_location=torch.device('cpu'))
        self.model.load_state_dict(checkpoint)
        self.model.eval()

    def output(self, input_arr):
        input_arr = np.expand_dims(input_arr,axis=0)
        input_arr = torch.from_numpy(input_arr).float()
        with torch.no_grad():
            probs = torch.squeeze(F.softmax(self.model(input_arr), dim=1)).numpy()

        return probs

    def output_max(self, input_arr):
        input_arr = np.expand_dims(input_arr,axis=0)
        input_arr = torch.from_numpy(input_arr).float()
        with torch.no_grad():
            output = np.argmax(torch.squeeze(F.softmax(self.model(input_arr), dim=1)).numpy())

        return output

if __name__ == "__main__":
    result={}
    num=int(input())
    afile_name,pfile_name = batch_showname(path)
    Pfile,Afile,Afile_se,e_latlon = apfile_read(pfile_name[num],afile_name[num])
    data_factor,sta_lon_lat = factor()
    Afile_se_data = cal_factor(Afile_se,data_factor)
    Afile_se_5s,sta_P = cut_st(Afile_se_data,Pfile)
    Afile_se_5s_data_123 = append_array(Afile_se_5s,sta_P)
    pga_data = data_pga(Afile_se_5s_data_123,Pfile)
    
    for i,j in Afile_se_5s_data_123.items():
        input_arr=j
        model_cnn1 = model_cnn(model=model_cnn_cls.CNN5(), param_path="train_data_pga80_snr5_update_500_label4_model.ckpt")
        Probability = model_cnn1.output(input_arr)
        Predict = model_cnn1.output_max(input_arr)
        result[i]=[sta_lon_lat[i][0],sta_lon_lat[i][1],Probability[0],Probability[1],Predict,pga_data[i]]
        print("station:",i)
        print("Probability:", Probability)
        print("Predict:", Predict)
        print("-"*40)

    result_txt(Pfile,afile_name[num],result)
    data_df_TN,data_df_FN,data_df_TP,data_df_FP = data_read(afile_name[num],result,e_latlon)
    plot_result(data_df_TN,data_df_FN,data_df_TP,data_df_FP,e_latlon,afile_name[num])

0


/home/sysop/anaconda3/envs/eqt/lib/python3.7/site-packages/ipykernel-6.0.2-py3.7.egg/ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


station: ALS
Probability: [ 0.0723011  0.9276989]
Predict: 1
----------------------------------------
station: ANP
Probability: [ 0.97080606  0.02919391]
Predict: 0
----------------------------------------
station: BAC
Probability: [ 0.96047586  0.03952412]
Predict: 0
----------------------------------------
station: CHK
Probability: [ 0.00488757  0.9951125 ]
Predict: 1
----------------------------------------
station: CHN1
Probability: [ 0.9115486  0.0884514]
Predict: 0
----------------------------------------
station: CHN2
Probability: [ 0.5388211   0.46117896]
Predict: 0
----------------------------------------
station: CHN3
Probability: [ 0.9488953  0.0511047]
Predict: 0
----------------------------------------
station: CHN4
Probability: [ 0.6364231   0.36357686]
Predict: 0
----------------------------------------
station: CHN5
Probability: [ 0.04121041  0.9587895 ]
Predict: 1
----------------------------------------
station: CHN8
Probability: [ 0.93509036  0.06490964]
Predict: 0
-

station: WDG
Probability: [ 0.9686868   0.03131313]
Predict: 0
----------------------------------------
station: WDL
Probability: [ 0.4102703   0.58972967]
Predict: 1
----------------------------------------
station: WGK
Probability: [ 0.50908756  0.49091247]
Predict: 0
----------------------------------------
station: WHF
Probability: [ 0.523216    0.47678402]
Predict: 0
----------------------------------------
station: WLC
Probability: [ 0.969729    0.03027095]
Predict: 0
----------------------------------------
station: WML
Probability: [ 0.8975715   0.10242853]
Predict: 0
----------------------------------------
station: WNT
Probability: [ 0.40787116  0.5921288 ]
Predict: 1
----------------------------------------
station: WNT1
Probability: [ 0.59177095  0.40822896]
Predict: 0
----------------------------------------
station: WPL
Probability: [ 0.9110702   0.08892978]
Predict: 0
----------------------------------------
station: WSF
Probability: [ 0.95541817  0.04458179]
Predict: 0
